 # Setup

In [1]:
from datetime import datetime, timedelta
import requests
import time
# elsevier package
# import elsapy

import numpy as np
import bs4 as beutifulsoup
import urllib
from robobrowser import RoboBrowser

from bs4 import BeautifulSoup
from urllib.request import urlopen

# Dark Sky forecast API wrapper
import forecastio

#  geocoder
from geopy.geocoders import Nominatim

# from selenium import webdriver
import re as re
# import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
# %matplotlib inline

# Gameplan

### I spend a lot of time looking at forecasts as I plan outdoorsy trips, so here I'm going code a little bit of that work. I pulled the list of top mountain bike trails in california, fetched their coordinates, then fetched the weather at those places at a future time, and finally sorted by a weather feature (cloudiness because everything else took too long). Neato!

## Get the trails and cities 

In [2]:
URL = 'https://www.singletracks.com/mountain-bike/best_trails.php?new_state_id=385'

In [3]:
# get it and make it look good
url = urlopen(URL)
soup = BeautifulSoup(url.read(), 'html.parser')
topmtb = soup.prettify()
# print(topmtb)

In [4]:
# get the trail name and nearest city
pattern = re.compile('(?P<trail_name>.*)\n\s\s</a>\n\s</td>\n.*\n(?P<city>.*)')
# was trying to return the same but without whitespace before the cities and names
# this worked on pythex.org but returned nothing here :( 
# maybe it doesn't matter??
# pattern = re.compile('(?P<trail_name>^\S*.*)\n\s\s</a>\n\s</td>\n.*\n\s*(?P<city>\S.*)')

match_iterator = pattern.finditer(topmtb)
for match in match_iterator:
    print(match.groupdict())

{'trail_name': '   Downieville Downhill', 'city': '  Downieville'}
{'trail_name': '   Forest Of Nisene Marks and Soquel Demonstration Forest', 'city': '  Santa Cruz'}
{'trail_name': '   Northstar Bike Park', 'city': '  Truckee'}
{'trail_name': '   Tahoe Rim Trail', 'city': '  Tahoe City'}
{'trail_name': '   Clementine / Forresthill Connector Trail', 'city': '  Auburn'}
{'trail_name': '   Camp Tamarancho', 'city': '  Marin'}
{'trail_name': '   San Juan Trail', 'city': '  San Juan Capistrano'}
{'trail_name': '   Hole In The Ground', 'city': '  Truckee'}
{'trail_name': '   Noble Canyon', 'city': '  San Diego'}
{'trail_name': "   Mr. Toad's Wild Ride", 'city': '  South Lake Tahoe'}
{'trail_name': '   Holy Jim Trail', 'city': '  Trabuco Canyon'}
{'trail_name': '   Mount Lowe Railway', 'city': '  Pasadena'}
{'trail_name': '   Clementine Loop', 'city': '  Auburn'}
{'trail_name': '   Hummingbird', 'city': '  Simi Hills'}
{'trail_name': '   1st / 2nd / 3rd Divide', 'city': '  Downieville'}
{'tr

In [5]:
# format the dict as a df
match_list = []
match_iterator = pattern.finditer(topmtb)
for match in match_iterator:
    match_list.append(match.groupdict())

df = pd.DataFrame(match_list)
# probably didn't need to do this but i'm so used to df's.... 

## Get the city coordinates

In [6]:
# check that I can get the lat,lon from a given place
geolocator = Nominatim()
# location = geolocator.geocode("South Lake Tahoe, California")
location = geolocator.geocode("Auburn")
print(location.address)
print((location.latitude, location.longitude))

# Wrong Auburn! Need to add the state when I input city 
# also notice one city is spelled wrong so I can't fetch its lat,lon

Auburn, Lee County, Alabama, United States of America
(32.527002, -85.4367484053398)


In [ ]:
# print(location.raw)

In [7]:
# add the lat and lon as new columns in my df
# seems like a good opportunity to use df.apply and practice writing functions

# write  functions that will take the city from the df as input and return
# the lat and lon 
def latify(place):
    location = geolocator.geocode(place+', CA', timeout=5)
    time.sleep(1)
    # really wish I had included exception catcher sooner...
    if hasattr(location, 'latitude'):
        # also wish I had known 'return object.attribute' is a no go
        out = getattr(location, 'latitude')
        return out
    else:
        return float('NaN')
    
def lonify(place):
    location = geolocator.geocode(place+', CA',timeout=5)
    time.sleep(1)
    if hasattr(location, 'longitude'):
        out = getattr(location, 'longitude')
        return out
    else:
        return float('NaN')

In [11]:
# apply the two functions to the df and add the lat, lon for each city
# df['lat'] = df['city'].apply(latify)
df['lon'] = df['city'].apply(lonify)

In [12]:
#check it
df

,city,trail_name,lat,lon
0,Downieville,Downieville Downhill,39.560444,-120.828218
1,Santa Cruz,Forest Of Nisene Marks and Soquel Demonstra...,40.213924,-8.433509
2,Truckee,Northstar Bike Park,39.327962,-120.183253
3,Tahoe City,Tahoe Rim Trail,39.168937,-120.143479
4,Auburn,Clementine / Forresthill Connector Trail,45.017774,-64.868742
5,Marin,Camp Tamarancho,38.040914,-122.619964
6,San Juan Capistrano,San Juan Trail,33.501693,-117.662551
7,Truckee,Hole In The Ground,39.327962,-120.183253
8,San Diego,Noble Canyon,32.717421,-117.162771
9,South Lake Tahoe,Mr. Toad's Wild Ride,38.933241,-119.984348


## Now get the forecasts for the locations ID'd by coordinates

In [13]:
#check if it works
api_key = "458030bd4d05113d5ca84e56abe8fb8f" # https://darksky.net/dev
forecast = forecastio.load_forecast(api_key, df.lat[1], df.lon[1])
byHour = forecast.hourly()
print(byHour.summary)
# woohoo! cloudy tho

Partly cloudy starting later tonight, continuing until tomorrow morning.


In [389]:
# define a function that will call a forecastio functoion to retrieve weather data,
# then put some of the data in the df
# def addcloudcover(lat,lon,key):
#     future = datetime.now() + timedelta(hours=delta)
#     forecast = forecastio.load_forecast(key,lat,lon,future)
#     out = forecast.json['currently']['cloudCover']
#     return out

# I intended to apply this function to the lat,lon columns of the df, only to find out
# that that's easier said than done. I deleted my various attempts in frustration

In [14]:
#desperation for loop approach
lats = np.asarray(df['lat'])
lons = np.asarray(df['lon'])

future = datetime.now() + timedelta(hours=4)
cloudy = []
for icloud in range(0,75):
    if np.isnan(lats[icloud]):
        cloudy.append('NaN')
        continue
    forecast = forecastio.load_forecast(api_key,lats[icloud],lons[icloud],future)
    out = forecast.json['currently']['cloudCover']
    cloudy.append(out)
    

In [15]:
cloudy = np.asarray(cloudy)
df['cloud_cover'] = pd.Series(cloudy, index=df.index)
# add column to df


In [16]:
# sort by cloudiness
df.sort_values(by=['cloud_cover'])
# Looks like Santa Cruz is the place to be! 

,city,trail_name,lat,lon,cloud_cover
47,Frazier Park,McGill Trail,34.811555,-118.957763,0
13,Simi Hills,Hummingbird,34.278336,-118.734536,0
69,Tehachapi,NICA Loop,35.132188,-118.448974,0
67,Hesperia,Green Trail,34.426389,-117.300878,0
24,Tehachapi,TMTA Lehigh trails,35.132188,-118.448974,0
66,Pine Valley,Mount Laguna Recreation Area,56.016667,-120.616667,0
20,Cordelia,Rockville Park,38.210471,-122.135801,0.01
26,Agoura Hills,Cheeseboro / Palo Comado Canyon,34.136395,-118.774535,0.01
27,Fairfax,Solstice,44.420933,-76.062431,0.01
19,Santa Clarita,Viper,34.391664,-118.542586,0.01


In [20]:
forecast.json.items

AttributeError: 'dict' object has no attribute 'dict'

In [30]:
for key, value in forecast.json.items():
    print (key, value,  ')

latitude 45.0177739 

longitude -64.8687417 

timezone America/Moncton 

currently {'time': 1526596566, 'summary': 'Mostly Cloudy', 'icon': 'partly-cloudy-day', 'precipIntensity': 0.0178, 'precipProbability': 0.02, 'precipType': 'rain', 'temperature': 13.98, 'apparentTemperature': 13.98, 'dewPoint': 9.3, 'humidity': 0.73, 'pressure': 1017.53, 'windSpeed': 9.08, 'windGust': 19.79, 'windBearing': 358, 'cloudCover': 0.67, 'uvIndex': 0, 'visibility': 16.09, 'ozone': 326.43} 

hourly {'summary': 'Partly cloudy until night.', 'icon': 'partly-cloudy-day', 'data': [{'time': 1526526000, 'summary': 'Mostly Cloudy', 'icon': 'partly-cloudy-night', 'precipIntensity': 0.0102, 'precipProbability': 0.03, 'precipType': 'rain', 'temperature': 7.59, 'apparentTemperature': 5.71, 'dewPoint': 0.63, 'humidity': 0.61, 'pressure': 1021, 'windSpeed': 10.32, 'windGust': 29.05, 'windBearing': 202, 'cloudCover': 0.74, 'uvIndex': 0, 'visibility': 16.09, 'ozone': 327.72}, {'time': 1526529600, 'summary': 'Partly Clou

In [34]:
import math
math.sqrt(15000)

122.47448713915891